In [13]:
# load in csvOut.csv data (holds image name & classification)
import csv
import os #to get all picture in directory
from PIL import Image #to open images
import numpy as np #load in numpy module
import sys

# loads in full csvOut data (name & classification)
with open('..//..//annotations_handheld_exclude_test.csv', newline='') as csvfile:
    cls_full = list(csv.reader(csvfile))

In [ ]:
# Get the list of all files in directory
path = "..//..//images_handheld_resized"
dir_list = os.listdir(path)

# Initialize dictionaries to hold bitmap information for each image
rgb = {}
rgb_list = []
cls = []

# loop through files and get bit map for each (save as object where filename => bitmap for r,g,b)
for file in cls_full:
    
    # push classification to list
    cls.append(file[1])
    
    # method found https://stackoverflow.com/questions/46385999/transform-an-image-to-a-bitmap
    img = Image.open(path + "\\" + file[0])
    
    # resize image
    #small_img = img.resize((600, 400))
    #smaller_img = img.resize((300, 200))
    smallest_img = img.resize((120, 80))
    #test = img.resize((90, 60))
    ary = np.array(smallest_img)
    
    # Save all 3 together
    rgb[file[0]] = ary
    
    # Split the three channels
    #r[file], g[file], b[file] = np.split(ary,3,axis=2)
    
    # flatten out to 1 array
    rgb[file[0]]=rgb[file[0]].reshape(-1)
        
    # loop through bitmap and group into values by 5s
    for i in range(len(rgb[file[0]])):
        if (rgb[file[0]][i] < 61):
            rgb[file[0]][i] = 0
        elif (rgb[file[0]][i] < 214):
            rgb[file[0]][i] = 1
        else:
            rgb[file[0]][i] = 2
    
    rgb_list.append(rgb[file[0]])
    
    # convert array to string
    #rgb_string[file] = np.array2string(rgb[file], precision=2, separator=',', suppress_small=True)


In [3]:
rgb['DSC00026.JPG']

array([0, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [ ]:
# convert bitmaps to indexed np array
rgb_arr = np.array([rgb[row] for row in rgb])
rgb_arr2 = np.array(rgb_list)

# convert classifications to numpy array
cls_np = np.array(cls)

In [ ]:
# Training model (load necessary libraries)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(rgb_arr2, cls_np, test_size=0.2)

In [ ]:
from sklearn.metrics import accuracy_score

svc = svm.SVC(C=0.1, gamma=0.0001, kernel='poly', probability=True)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [7]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(svm.SVC(C=0.1, gamma=0.0001, kernel='poly', probability=True), X_train, y_train, cv=3)
accuracies

array([0.58733624, 0.56892779, 0.57330416])